#Emirhan Aykan

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
import spacy
import html

In [15]:
mails = pd.read_csv('/content/drive/MyDrive/kluza/all_mails.csv', header=0)
keywords = pd.read_csv('/content/drive/MyDrive/kluza/keywords.csv', header=0)
metalabel = pd.read_csv('/content/drive/MyDrive/kluza/metalabel.csv', header=0)

In [16]:
mails.head()

,Unnamed: 0,id,author,subject,date,content_url,dtype,content
0,0,1,Daniel Fullarton (JIRA),[jira] [Created] (CAMEL-11099) Unhandled Cla...,"Sun, 02 Apr, 23:59",http://mail-archives.apache.org/mod_mbox/camel...,issues,Daniel Fullarton created CAMEL-11099:&#010;---...
1,1,2,ASF GitHub Bot (JIRA),[jira] [Commented] (CAMEL-11099) Unhandled C...,"Mon, 03 Apr, 00:01",http://mail-archives.apache.org/mod_mbox/camel...,issues,&#010; [ https://issues.apache.org/jira/bro...
2,2,3,Zoran Regvart (JIRA),[jira] [Resolved] (CAMEL-11099) Unhandled Cl...,"Tue, 04 Apr, 13:33",http://mail-archives.apache.org/mod_mbox/camel...,issues,&#010; [ https://issues.apache.org/jira/br...
3,3,4,ASF GitHub Bot (JIRA),[jira] [Commented] (CAMEL-11099) Unhandled C...,"Wed, 05 Apr, 01:38",http://mail-archives.apache.org/mod_mbox/camel...,issues,&#010; [ https://issues.apache.org/jira/bro...
4,4,5,NaN,[jira] [Commented] (CAMEL-11092) If setting Ex...,NaN,http://mail-archives.apache.org/mod_mbox/camel...,issues,&#010; [ https://issues.apache.org/jira/bro...


In [21]:
def clear_text(text):
  cleared_text = html.unescape(text)
  cleared_text = cleared_text.split('\n>')[0]
  cleared_text = cleared_text.replace('\n', '')
  return cleared_text

In [22]:
mails['content'] = mails['content'].apply(clear_text)

In [27]:
mails['content'][125]

'    [ https://issues.apache.org/jira/browse/CAMEL-11111?page=com.atlassian.jira.plugin.system.issuetabpanels:comment-tabpanel&focusedCommentId=15961981#comment-15961981] ASF GitHub Bot commented on CAMEL-11111:----------------------------------------GitHub user scranton opened a pull request:    https://github.com/apache/camel/pull/1608    CAMEL-11111: implemented throwExceptionOnFailure Producer attribute    You can merge this pull request into a Git repository by running:    $ git pull https://github.com/scranton/camel CAMEL-11111Alternatively you can review and apply these changes as the patch at:    https://github.com/apache/camel/pull/1608.patchTo close this pull request, make a commit to your master/trunk branchwith (at least) the following in the commit message:    This closes #1608    ----commit eb4a236e36c4665e6234d72db40d8c7ef4e5d6fbAuthor: Scott Cranton <scott@cranton.com>Date:   2017-04-08T22:42:46Z    CAMEL-11111: implemented throwExceptionOnFailure attribute for Producer

In [18]:
mails['date'][50]

'Tue, 18 Apr, 07:44'

In [52]:
mails['content_url'][52]

'http://mail-archives.apache.org/mod_mbox/camel-issues/201704.mbox/ajax/%3CJIRA.13061376.1491296412000.198854.1491296449501%40Atlassian.JIRA%3E'

In [38]:
import spacy

nlp = spacy.load("en_core_web_sm")

def extract_ocel(email_content, timestamp):
    # Process email content using spaCy
    doc = nlp(email_content)

    # Extract the event type based on verb phrases
    event_type = ""
    for sent in doc.sents:
        for token in sent:
            if token.pos_ == "VERB":
                event_type = token.text
                break
        if event_type:
            break

    # Extract the actor based on named entities
    actor = ""
    for entity in doc.ents:
        if entity.label_ == "PERSON":
            actor = entity.text
            break

    # Extract the object based on the issue/task ID
    object_match = re.search(r'CAMEL-\d+', email_content)
    object_name = object_match.group() if object_match else ""

    # Return the extracted OCEL
    return {
        "Event Type": event_type,
        "Actor": actor,
        "Timestamp": timestamp,
        "Object": object_name
    }

In [48]:
# Example usage for a single email and timestamp
email_content = mails['content'][2]
timestamp = mails['date'][2]

In [44]:
ocel = extract_ocel(email_content, timestamp) # 100
print(ocel)
# Work on CAMEL-11108 started by Luca Burgazzoli.

{'Event Type': 'started', 'Actor': '', 'Timestamp': 'Thu, 06 Apr, 11:54', 'Object': 'CAMEL-11108'}


In [49]:
ocel = extract_ocel(email_content, timestamp) # 100
print(ocel)

{'Event Type': 'https://issues.apache.org/jira/browse/CAMEL-11099?page=com.atlassian.jira.plugin.system.issuetabpanels:all-tabpanel]Zoran', 'Actor': 'https://issues.apache.org/jira/browse/CAMEL-11099?page=com.atlassian.jira.plugin.system.issuetabpanels:all-tabpanel]Zoran Regvart', 'Timestamp': 'Tue, 04 Apr, 13:33', 'Object': 'CAMEL-11099'}


In [47]:
mails['content_url'][4]

'http://mail-archives.apache.org/mod_mbox/camel-issues/201704.mbox/ajax/%3CJIRA.13061008.1491177558000.206772.1491356321613%40Atlassian.JIRA%3E'